In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
from pyspark.sql.types import *

list=[[15224686, 'Green Dragon', 'The film follows',['Indie','Drama']],
      [15585766, 'The Rats of Tobruk', 'Three friends are',['Drama']]]


customSchema= StructType([StructField("movie_id", IntegerType()),
                        StructField("movie_name", StringType()),
                        StructField("plot", StringType()),
                          StructField("genre", ArrayType(StringType()))])

df=spark.createDataFrame(list,customSchema)

df.show()

+--------+------------------+-----------------+--------------+
movie_id| movie_name| plot| genre|
+--------+------------------+-----------------+--------------+
15224686| Green Dragon| The film follows|[Indie, Drama]|
15585766|The Rats of Tobruk|Three friends are| [Drama]|
+--------+------------------+-----------------+--------------+

In [3]:
list=[["['Indie', 'Drama', 'Action']"]]

df=spark.createDataFrame(list,['genre'])

display(df)

genre
"['Indie', 'Drama', 'Action']"


In [4]:
df.withColumn("genre", F.split(F.regexp_replace("genre", "\[|]| |'", ""),",")).show(truncate=False)

+----------------------+
genre |
+----------------------+
[Indie, Drama, Action]|
+----------------------+

In [5]:
list=[[0,[1,2]],
      [1,[0,1]],
      [2,[0,1]],
      [3,[0,1]]]

df=spark.createDataFrame(list,['id','label'])


df.show()

+---+------+
 id| label|
+---+------+
 0|[1, 2]|
 1|[0, 1]|
 2|[0, 1]|
 3|[0, 1]|
+---+------+

In [6]:
labels=['0','1','2']

from pyspark.sql import functions as F
df.withColumn("struct", F.struct(*[(F.struct(F.expr("size(filter(label,x->x={}))"\
                                                    .format("'"+y+"'"))).alias(y)) for y in labels]))\
            .select("id",*[F.col("struct.{}.col1".format(x)).alias('label'+x) for x in labels]).show()

+---+------+------+------+
 id|label0|label1|label2|
+---+------+------+------+
 0| 0| 1| 1|
 1| 1| 1| 0|
 2| 1| 1| 0|
 3| 1| 1| 0|
+---+------+------+------+

In [7]:
list=[[['a', 'b', 'b', 'c']],
     [['b', 'c', 'd']]]

df=spark.createDataFrame(list,['atr_list'])

df.show()


+------------+
 atr_list|
+------------+
[a, b, b, c]|
 [b, c, d]|
+------------+

In [8]:
from pyspark.sql import functions as F

a=df.withColumn("atr", F.expr("""transform(array_distinct(atr_list),x->aggregate(atr_list,0,(acc,y)->\
                               IF(y=x, acc+1,acc)))"""))\
  .withColumn("zip", F.explode(F.arrays_zip(F.array_distinct("atr_list"),("atr"))))\
  .select("zip.*").withColumnRenamed("0","elements")\
  .groupBy("elements").agg(F.sum("atr").alias("sum"))\
  .collect()

{a[i][0]: a[i][1] for i in range(len(a))} 

Out[37]: {'d': 1, 'c': 2, 'b': 3, 'a': 1}

In [9]:
df.withColumn("atr", F.expr("""transform(array_distinct(atr_list),x->aggregate(atr_list,0,(acc,y)->\
                               IF(y=x, acc+1,acc)))"""))\
  .withColumn("elements", F.array_distinct("atr_list"))\
  .withColumn("atr1", F.expr("""transform(arrays_zip(elements,atr),x-> array(x.atr,x.elements))"""))\
  .withColumn("atr1", F.expr("""transform(atr1, x-> struct(x[0] as aggregate(atr,0,(acc,y)->IF(y==x[1],acc=x[1],acc))))"""))\
  .show(truncate=False)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling z:org.apache.spark.sql.functions.expr.
: org.apache.spark.sql.catalyst.parser.ParseException: 
mismatched input '(' expecting {')', ','}(line 1, pos 44)

== SQL ==
transform(atr1, x-> struct(x[0] as aggregate(atr,0,(acc,y)->IF(y==x[1],acc=x[1],acc))))
--------------------------------------------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:241)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:117)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:55)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parseExpression(ParseDriver.scala:44)
	at com.databricks.sql.parser.DatabricksSqlParser.parseExpression(DatabricksSqlParser.scala:46)
	at org.apache.spark.sql.functions$.expr(functions.scala:1366)
	at org.apache.spark.sql.functions.expr(functions.scala)
	at sun.reflect.GeneratedMethodAccessor347.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 ParseException Traceback (most recent call last)
 <command-1896245210851985> in <module> 
 3 . withColumn ( "elements" , F . array_distinct ( "atr_list" ) ) \ 
 4 . withColumn ( "atr1" , F . expr ( """transform(arrays_zip(elements,atr),x-> array(x.atr,x.elements))""" ) ) \ 
 ----> 5 . withColumn ( "atr1" , F . expr ( """transform(atr1, x-> struct(x[0] as aggregate(atr,0,(acc,y)->IF(y==x[1],acc=x[1],acc))))""" ) ) \ 
 6 . show ( truncate = False ) 

 /databricks/spark/python/pyspark/sql/functions.py in expr (str) 
 673 """
 674 sc = SparkContext . _active_spark_context
 --> 675 return Column ( sc . _jvm . functions . expr ( str ) ) 
 676 
 677 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 72 if s . startswith ( 'org.apache.spark.sql.catalyst.parser.ParseException: ' ) : 
 ---> 73 raise ParseException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 74 if s . startswith ( 'org.apache.spark.sql.streaming.StreamingQueryException: ' ) : 
 75 raise StreamingQueryException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 ParseException : "\nmismatched input '(' expecting {')', ','}(line 1, pos 44)\n\n== SQL ==\ntransform(atr1, x-> struct(x[0] as aggregate(atr,0,(acc,y)->IF(y==x[1],acc=x[1],acc))))\n--------------------------------------------^^^\n"

In [10]:
for i in a:
  print(i[0],i[1])

d 1
c 2
b 3
a 1

In [11]:
list=[[[['01', '100.0'], ['01', '400.0'], [None, '0.0'], ['06', '0.0'], ['01', '400.0'], [None, '0.0'], ['06', '200.0']]],
      [[[None, '200.0'], ['06', '300.0'], ['01', '500'], ['06', '100.0'], ['01', '200'], ['07', '50.0']]]]


df=spark.createDataFrame(list,['arr'])

df.show(truncate=False)

+---------------------------------------------------------------------------------+
arr |
+---------------------------------------------------------------------------------+
[[01, 100.0], [01, 400.0], [, 0.0], [06, 0.0], [01, 400.0], [, 0.0], [06, 200.0]]|
[[, 200.0], [06, 300.0], [01, 500], [06, 100.0], [01, 200], [07, 50.0]] |
+---------------------------------------------------------------------------------+

In [12]:
df.withColumn("Max_01", F.expr("""array_max(transform(filter(arr, x-> exists(x,y-> y='01')),z-> z[1]))""").cast('float'))\
  .show(truncate=False)

+----------------------------------------------------------------------------------+------+
arr |Max_01|
+----------------------------------------------------------------------------------+------+
[[01, 100.0], [01, 400.0], [, 0.0], [06, 0.0], [01, 1000.0], [, 0.0], [06, 200.0]]|400.0 |
[[, 200.0], [06, 300.0], [01, 500], [06, 100.0], [01, 200], [07, 50.0]] |500.0 |
+----------------------------------------------------------------------------------+------+

In [13]:
from pyspark.sql import functions as F

df.withColumn("Max_01",F.when(F.size(F.expr("""filter(arr,x-> exists(x,y->y='01'))"""))!=0,
       F.expr("""array_max(transform(filter(arr, x-> exists(x,y-> y='01')),z-> float(z[1])))"""))\
             .otherwise(F.lit(0)))\
  .withColumn("Max_06",F.when(F.size(F.expr("""filter(arr,x-> exists(x,y->y='06'))"""))!=0,
       F.expr("""array_max(transform(filter(arr, x-> exists(x,y-> y='06')),z-> float(z[1])))"""))\
             .otherwise(F.lit(0)))\
  .withColumn("Max_07",F.when(F.size(F.expr("""filter(arr,x-> exists(x,y->y='07'))"""))!=0,
       F.expr("""array_max(transform(filter(arr, x-> exists(x,y-> y='07')),z-> float(z[1])))"""))\
             .otherwise(F.lit(0)))\
   .show(truncate=False)

#+---------------------------------------------------------------------------------+------+------+------+
#|arr                                                                              |Max_01|Max_06|Max_07|
#+---------------------------------------------------------------------------------+------+------+------+
#|[[01, 100.0], [01, 400.0], [, 0.0], [06, 0.0], [01, 400.0], [, 0.0], [06, 200.0]]|400.0 |200.0 |0.0   |
#|[[, 200.0], [06, 300.0], [01, 500], [06, 100.0], [01, 200], [07, 50.0]]          |500.0 |300.0 |50.0  |
#+---------------------------------------------------------------------------------+------+------+------+


+---------------------------------------------------------------------------------+------+------+------+
arr |Max_01|Max_06|Max_07|
+---------------------------------------------------------------------------------+------+------+------+
[[01, 100.0], [01, 400.0], [, 0.0], [06, 0.0], [01, 400.0], [, 0.0], [06, 200.0]]|400.0 |200.0 |0.0 |
[[, 200.0], [06, 300.0], [01, 500], [06, 100.0], [01, 200], [07, 50.0]] |500.0 |300.0 |50.0 |
+---------------------------------------------------------------------------------+------+------+------+

In [14]:
  .withColumn("Max_06",F.expr("""array_max(transform(filter(arr, x-> exists(x,y-> y='06')),z-> float(z[1])))"""))\
  .withColumn("Max_07",F.expr("""array_max(transform(filter(arr, x-> exists(x,y-> y='07')),z-> float(z[1])))"""))\
  .show()

In [15]:
list=[[1 , False ,'2020-05-01'],
      [1  , True ,'2020-05-06'],
      [2   ,True ,'2020-05-04'],
      [2  ,False ,'2020-05-07'],
      [2   ,True ,'2020-05-09'],
      [3  ,False ,'2020-05-11']]

df=spark.createDataFrame(list,['id','valid','eventdate'])

df.show()

+---+-----+----------+
 id|valid| eventdate|
+---+-----+----------+
 1|false|2020-05-01|
 1| true|2020-05-06|
 2| true|2020-05-04|
 2|false|2020-05-07|
 2| true|2020-05-09|
 3|false|2020-05-11|
+---+-----+----------+

In [16]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("id").orderBy(F.to_date("eventdate","yyyy-MM-dd"))

df.withColumn("lead", F.lead("eventdate").over(w))\
  .withColumn("sequence", F.when(F.col("lead").isNotNull(),
                                 F.expr("""sequence(to_date(eventdate),date_sub(to_date(lead),1), interval 1 day)"""))\
                                 .otherwise(F.array("eventdate")))\
 .select("id","valid",F.explode("sequence").alias("eventdate"))\
 .show(truncate=False)

+---+-----+----------+
id |valid|eventdate |
+---+-----+----------+
1 |false|2020-05-01|
1 |false|2020-05-02|
1 |false|2020-05-03|
1 |false|2020-05-04|
1 |false|2020-05-05|
1 |true |2020-05-06|
3 |false|2020-05-11|
2 |true |2020-05-04|
2 |true |2020-05-05|
2 |true |2020-05-06|
2 |false|2020-05-07|
2 |false|2020-05-08|
2 |true |2020-05-09|
+---+-----+----------+

In [17]:
list=[['Xxxxx998',    None  , '01'   ,   '04'    ,  2000    ,1000,    100],     
      ['Xxxxx997' ,   '01'      ,'01'    ,  '07'     , 200    , 300  ,   400],    
      ['Xxxxx996'  ,  '07'      ,None  ,  None   , 100  ,   None ,   None],   
      ['Xxxx910'    , None    ,None  ,  None  , 300,     100  ,   200]] 

df=spark.createDataFrame(list,['Primary_id','Code_1','Code_2','Code_3','Amt_1','Amt_2','Amt_3'])

df.show()

+----------+------+------+------+-----+-----+-----+
Primary_id|Code_1|Code_2|Code_3|Amt_1|Amt_2|Amt_3|
+----------+------+------+------+-----+-----+-----+
 Xxxxx998| null| 01| 04| 2000| 1000| 100|
 Xxxxx997| 01| 01| 07| 200| 300| 400|
 Xxxxx996| 07| null| null| 100| null| null|
 Xxxx910| null| null| null| 300| 100| 200|
+----------+------+------+------+-----+-----+-----+

In [18]:
from pyspark.sql import functions as F

dictionary = dict(zip(['Code_1','Code_2','Code_3'], ['Amt_1','Amt_2','Amt_3']))

df.withColumn("trial", F.array(*[F.array(F.col(x),F.col(y).cast("string"))\
                                          for x,y in dictionary.items()]))\
  

+----------+------+------+------+-----+-----+-----+------+------+------+
Primary_id|Code_1|Code_2|Code_3|Amt_1|Amt_2|Amt_3|Max_01|Max_07|Max_06|
+----------+------+------+------+-----+-----+-----+------+------+------+
Xxxxx998 |null |01 |04 |2000 |1000 |100 |1000 |0 |0 |
Xxxxx997 |01 |01 |07 |200 |300 |400 |300 |400 |0 |
Xxxxx996 |07 |null |null |100 |null |null |0 |100 |0 |
Xxxx910 |null |null |null |300 |100 |200 |0 |0 |0 |
+----------+------+------+------+-----+-----+-----+------+------+------+

In [19]:
from pyspark.sql import functions as F

dictionary = dict(zip(['Code_1','Code_2','Code_3'], ['Amt_1','Amt_2','Amt_3']))

df.withColumn("trial", F.array(*[F.array(F.col(x),F.col(y).cast("string"))\
                                          for x,y in dictionary.items()]))\
  .withColumn("Max_01",F.when(F.size(F.expr("""filter(trial,x-> exists(x,y->y='01'))"""))!=0,\
       F.expr("""array_max(transform(filter(trial, x-> exists(x,y-> y='01')),z-> float(z[1])))"""))\
             .otherwise(F.lit(0)))\
  .withColumn("Max_06",F.when(F.size(F.expr("""filter(trial,x-> exists(x,y->y='06'))"""))!=0,\
       F.expr("""array_max(transform(filter(trial, x-> exists(x,y-> y='06')),z-> float(z[1])))"""))\
             .otherwise(F.lit(0)))\
  .withColumn("Max_07",F.when(F.size(F.expr("""filter(trial,x-> exists(x,y->y='07'))"""))!=0,\
       F.expr("""array_max(transform(filter(trial, x-> exists(x,y-> y='07')),z-> float(z[1])))"""))\
             .otherwise(F.lit(0)))\
  .drop("trial").show(truncate=False)

+----------+------+------+------+-----+-----+-----+------+------+------+
Primary_id|Code_1|Code_2|Code_3|Amt_1|Amt_2|Amt_3|Max_01|Max_06|Max_07|
+----------+------+------+------+-----+-----+-----+------+------+------+
Xxxxx998 |null |01 |04 |2000 |1000 |100 |1000.0|0.0 |0.0 |
Xxxxx997 |01 |01 |07 |200 |300 |400 |300.0 |0.0 |400.0 |
Xxxxx996 |07 |null |null |100 |null |null |0.0 |0.0 |100.0 |
Xxxx910 |null |null |null |300 |100 |200 |0.0 |0.0 |0.0 |
+----------+------+------+------+-----+-----+-----+------+------+------+

In [20]:
Primary_id  Code_1  Code_2  Code_3  Amt_1   Amt_2   Amt_3   Max_01  Max_07  Max_06
Xxxxx998    Null    01      04      2000    1000    100     1000    0       0
Xxxxx997    01      01      07      200     300     400     300     400     0
Xxxxx996    07      Null    Null    100     Null    Null    100     0       0
Xxxx910     Null    Null    Null    300     100     200     0       0       0